In [ ]:
import argparse
import networkx as nx
import numpy as np
import scipy as sp
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import my

In [ ]:
args = argparse.Namespace()
args.depth = 10
args.graph = 'soc-Epinions1-reduced'
args.n_features = 8
args.n_machines = 10
args.radius = 3

In [ ]:
class Objective:
    def __init__(self, g, n_machines):
#         number_of_nodes = max(tuple(g.nodes())) + 1
        s = sp.sparse.lil_matrix((g.number_of_nodes(), g.number_of_edges()))
        for i, (j, k) in enumerate(g.edges()):
            s[j, i] = 1
            s[k, i] = 1
        s_coo = s.tocoo()
        rows = th.from_numpy(s_coo.row).long().view(1, -1)
        cols = th.from_numpy(s_coo.col).long().view(1, -1)
        data = th.from_numpy(s_coo.data).float()
        self.s = th.sparse.FloatTensor(th.cat((rows, cols), 0), data, s_coo.shape)
        self.n_machines = n_machines
        
    def __call__(self, x):
        y = th.multinomial(x, num_samples=1)
        y = my.onehot(y, self.n_machines)
        y = th.mm(self.s, y)
        r = th.sum(y)
        p = (th.sum(y, 1) + 1) / r
        b = -p * th.log(p)
        objective = -(r + b) * th.log(x)
        return objective

In [ ]:
class GNNModule(nn.Module):
    def __init__(self, in_features, out_features, deg, adj, nonlinear):
        super().__init__()
        self.deg, self.u, self.adj = deg, u, adj
        new_linear = lambda: nn.Parameter(th.randn(in_features, out_features))
        self.alpha1, self.alpha2, self.alpha3 = new_linear(), new_linear(), new_linear()
        self.alpha4 = nn.ParameterList([new_linear() for a in adj])
        self.beta1, self.beta2, self.beta3 = new_linear(), new_linear(), new_linear()
        self.beta4 = nn.ParameterList([new_linear() for a in adj])
        self.bn_alpha, self.bn_beta = nn.BatchNorm2d(out_features), nn.BatchNorm2d(out_features)
        self.nonlinear = nonlinear
    
    def forward(self, x):
        deg = th.mm(self.deg, x)
        u = th.zeros_like(x) + th.mean(x, 1, keep_dim=True)
        adj = [th.mm(a, x) for a in self.adj]
        alpha = th.mm(self.alpha1, x) + th.mm(self.alpha1, deg) + th.mm(self.alpha2, u) + \
            sum(th.mm(alpha, a) for alpha, a in zip(self.alpha4, adj))
        alpha = self.bn_alpha(self.nonlinear(alpha))
        beta = th.mm(self.beta1, x) + th.mm(self.beta1, deg) + th.mm(self.beta2, u) + \
            sum(th.mm(beta, a) for beta, a in zip(self.beta4, adj))
        beta = self.bn_beta(beta)
        return alpha + beta

class EdgeLinear(nn.Module):
    def __init__(self, in_features, out_features, adj):
        super().__init__()
        self.in_features, self.out_features = in_features, out_features
        self.adj = adj
    
    def forward(self, x):
        pass

class GNN(nn.Module):
    def __init__(self, features, n_classes, deg, adj, radius, nonlinear):
        a, adj_list = adj, []
        for i in range(radius):
            adj_list.append(a)
            a = th.mm(a, a)
        # TODO nn.Sequential
        self.module_list = nn.ModuleList([GNNModule(m, n, deg, u, adj_list, nonlinear)
                                          for m, n in zip(features[:-1], features[1:])])
        self.linear = EdgeLinear(features[-1], k, adj)
    
    def forward(self, x):
        for module in self.module_list:
            x = module(x)
        x = F.softmax(x, 1)
        return x

In [ ]:
g = my.read_edgelist(args.graph)
adj = nx.adj_matrix(g)
deg = sp.sum(adj, 1)
objective = Objective(g, 10)

In [ ]:
gnn = GNN((1,) + (args.n_features,) * args.depth, args.n_machines, deg, adj, args.radius, F.relu)